In [1]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from numpy import linspace, array, logspace, sin, cos, pi, arange, sqrt, arctan2, arccos


import astropy.io.fits as pyfits



In [2]:
#Grab the existing table columns we want 
cat = Table.read('../3D_CMZ/LBV_plots_stuff/CMZ_cloud_catalogue_data.tex')
absorp_tab = Table.read('../3D_CMZ/LBV_plots_stuff/absorption_table_merged.tex')

cat_index = cat['index']
cloud_list = cat['cloud_name']
l = cat['l']
b = cat['b']
v = cat['v']
sigma = cat['sigma']
rad = cat['rad']
NF_decision = cat['NF_decision']
lb_pixel_mask = cat['lb_pixel_mask']

corr_coeff = cat['r_p_max']
flux_diff = cat['flux_diff_max']
flux_ratio = cat['flux_ratio_max']

absorp_value = absorp_tab['fraction_value_peak_mask']
absorp_index = absorp_tab['leaf_id']

In [3]:
cat

index,cloud_name,l,b,v,sigma,rad,absorp,r_p_mom0,r_s_mom0,r_p_max,r_s_max,flux_diff_mom0,flux_ratio_mom0,I_cloud_max,I_model_max,flux_diff_max,flux_ratio_max,ext_frac_max,lb_pixel_mask,NF_decision,min_flux_max
str3,str14,float64,float64,int64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,str3,float64
1,G359.475-0.044,-0.525,-0.044,-102,6,4.3,3,0.261,0.444,0.261,0.444,31.669,0.506,212.667,317.106,31.669,0.506,0.992,1,LN,82.971
2,G359.508-0.135,-0.492,-0.135,-56,9,8.7,3,0.384,0.397,0.384,0.397,-20.608,0.623,224.209,276.371,-20.608,0.623,0.815,1,N,117.934
3,G359.561-0.001,-0.439,-0.001,-90,8,2.7,0,0.222,0.325,0.222,0.325,-6.674,0.551,197.054,263.15,-6.674,0.551,0.997,1,U,106.365
4a,G359.595-0.223,-0.405,-0.223,-27,13,4.6,3,0.468,0.257,0.433,0.254,-27.454,0.569,180.619,232.561,-26.881,0.567,0.947,1,N,100.903
4b,G359.595-0.223,-0.405,-0.223,-20,8,4.6,3,0.316,0.329,0.262,0.101,-10.883,0.517,178.699,234.446,-25.555,0.559,0.932,0,LN,100.903
5,G359.608+0.018,-0.392,0.018,-78,5,2.0,1,0.183,0.262,0.183,0.262,-26.81,0.575,185.012,230.972,-26.81,0.575,0.997,1,LN,123.728
6a,G359.688-0.132,-0.312,-0.132,-29,5,5.6,0,0.655,0.694,0.621,0.621,-1.841,0.565,209.379,285.681,0.229,0.56,0.995,0,LN,153.898
6b,G359.688-0.132,-0.312,-0.132,-21,12,5.6,0,0.301,0.282,0.281,0.236,-8.637,0.579,202.07,259.951,-9.201,0.576,0.993,1,U,151.445
7a,G359.701+0.032,-0.299,0.032,-73,11,2.4,1,-0.53,-0.417,-0.535,-0.404,-45.38,0.658,205.206,233.126,-44.223,0.655,0.776,1,LN,154.551


In [4]:
###just need this file for the multi-comp clouds
large_cat = Table.read('../3D_CMZ/LBV_plots_stuff/CMZ_cloud_catalogue_data.csv')
large_cat_index = large_cat['index']
large_cat_v = large_cat['v']

In [5]:
### For the stddevs, we need to find them for both the single 
### and multi-component clouds separately 
### then combine into the table 

#Find single v component clouds
single_v = {x for x in list(cloud_list) if list(cloud_list).count(x) == 1}


sing_index =[]
for i in range(len(cloud_list)):
    if cloud_list[i] in single_v:
        sing_index.append(i)
        

#find indexes with multiple velocities 
dup = {x for x in list(cloud_list) if list(cloud_list).count(x) > 1}

dup_index =[]
for i in range(len(cloud_list)):
    if cloud_list[i] in dup:
        dup_index.append(i)
        
id_v_list = []
for i in range(len(large_cat_index)):
    id_v_list.append('{}_{}'.format(large_cat_index[i], large_cat_v[i]))

In [6]:
flux_ratio_std = []
flux_diff_std = []

for i in range(len(cat_index)):
    if cat_index[i] in cat_index[sing_index]:
        cloud = cloud_list[i]
        
        cloud_flux_diff = pyfits.open('../3D_CMZ/Cloud_masks/{}/{}_flux_diff.fits'.format(cloud, cloud))[0].data
        fdiff_std = np.abs(np.nanstd(cloud_flux_diff))
        flux_diff_std.append('{:.2f}'.format(np.round(fdiff_std,2)))

        cloud_flux_ratio = pyfits.open('../3D_CMZ/Cloud_masks/{}/{}_flux_ratio.fits'.format(cloud, cloud))[0].data
        fratio_std = np.abs(np.nanstd(cloud_flux_ratio))
        flux_ratio_std.append('{:.2f}'.format(np.round(fratio_std,2)))
        
    if cat_index[i] in cat_index[dup_index]:
        cloud_flux_diff = pyfits.open('../3D_CMZ/Sub_masks/{}/max_submask/{}_{}_flux_diff_maxcutout_isolated.fits'.format(large_cat_index[i], large_cat_index[i],large_cat_v[i] ))[0].data
        fdiff_std = np.abs(np.nanstd(cloud_flux_diff))
        flux_diff_std.append('{:.2f}'.format(np.round(fdiff_std,2)))

        cloud_flux_ratio = pyfits.open('../3D_CMZ/Sub_masks/{}/max_submask/{}_{}_flux_ratio_maxcutout_isolated.fits'.format(large_cat_index[i], large_cat_index[i],large_cat_v[i] ))[0].data
        fratio_std = np.abs(np.nanstd(cloud_flux_ratio))
        flux_ratio_std.append('{:.2f}'.format(np.round(fratio_std,2)))

/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [7]:
### absorption table only has the ids with measurement available

#create nan array with size equal to 'index'
absorp_arr = []

#fill in where indices match. Make nan where no match.
for i in cat_index:
    if i in absorp_index:
        if i =='16a': #16a is an empty mask. replace with nan
            absorp_arr.append(np.nan)
        else: 
            absorp_arr.append(absorp_value[np.where(absorp_index==str(i))[0][0]])
    else: 
        absorp_arr.append(np.nan)
        


In [8]:
### Construct new table 

synth_table = Table()
synth_table['leaf_id'] = cat_index
synth_table['cloud_name'] = cat['cloud_name']
synth_table['l'] = l
synth_table['b'] = b
synth_table['v'] = v
synth_table['sigma'] = sigma
synth_table['rad'] = rad
synth_table['corr_coeff'] = corr_coeff.round(2)
synth_table['flux_diff'] = flux_diff.round(2)
synth_table['flux_diff_stdv'] = flux_diff_std
synth_table['flux_ratio'] = flux_ratio.round(2)
synth_table['flux_ratio_stdv'] = flux_ratio_std
synth_table['absorp_value'] = absorp_arr
synth_table['NF_decision'] = NF_decision
synth_table['lb_pixel_mask'] = lb_pixel_mask

synth_table


leaf_id,cloud_name,l,b,v,sigma,rad,corr_coeff,flux_diff,flux_diff_stdv,flux_ratio,flux_ratio_stdv,absorp_value,NF_decision,lb_pixel_mask
str3,str14,float64,float64,int64,int64,float64,float64,float64,str6,float64,str4,float64,str3,int64
1,G359.475-0.044,-0.525,-0.044,-102,6,4.3,0.26,31.67,59.09,0.51,0.20,nan,LN,1
2,G359.508-0.135,-0.492,-0.135,-56,9,8.7,0.38,-20.61,56.22,0.62,0.19,nan,N,1
3,G359.561-0.001,-0.439,-0.001,-90,8,2.7,0.22,-6.67,23.73,0.55,0.09,2.15,U,1
4a,G359.595-0.223,-0.405,-0.223,-27,13,4.6,0.43,-26.88,73.95,0.57,0.30,nan,N,1
4b,G359.595-0.223,-0.405,-0.223,-20,8,4.6,0.26,-25.56,101.97,0.56,0.42,nan,LN,0
5,G359.608+0.018,-0.392,0.018,-78,5,2.0,0.18,-26.81,23.27,0.57,0.11,0.52,LN,1
6a,G359.688-0.132,-0.312,-0.132,-29,5,5.6,0.62,0.23,25.94,0.56,0.10,3.3,LN,0
6b,G359.688-0.132,-0.312,-0.132,-21,12,5.6,0.28,-9.2,29.84,0.58,0.10,2.28,U,1
7a,G359.701+0.032,-0.299,0.032,-73,11,2.4,-0.54,-44.22,32.34,0.66,0.13,0.36,LN,1


In [9]:
synth_table.write("/Users/danilipman/Documents/Research/UConn/CMZ_SYNTH/synth_table.tex", overwrite=True)

    


In [281]:
print(absorp_index[22], absorp_value[22])

17c 1.63
